# DSE Course 2, Lab 4: Practice with Data Manipulation

**Instructor**: Wesley Beckner

**Contact**: wesleybeckner@gmail.com

<br>

---

<br>

In this lab we will focus on data manipulation with numpy and pandas.

<br>

---




In [1]:
import numpy as np
import pandas as pd

In [2]:
status = pd.read_csv("https://raw.githubusercontent.com/wesleybeckner/"\
                     "ds_for_engineers/main/data/mill_tool_wear/train.csv")
tests = pd.DataFrame()

for i in range(1,19):
  if i < 10:
    df = pd.read_csv("https://raw.githubusercontent.com/wesleybeckner/"\
                   "ds_for_engineers/main/data/mill_tool_wear/"\
                   "experiment_0{}.csv".format(i))
  else:
    df = pd.read_csv("https://raw.githubusercontent.com/wesleybeckner/"\
                   "ds_for_engineers/main/data/mill_tool_wear/"\
                   "experiment_{}.csv".format(i))
  df["feedrate"] = status.loc[i-1,"feedrate"]
  df["clamp_pressure"] = status.loc[i-1,"clamp_pressure"]
  df["tool_condition"] = status.loc[i-1,"tool_condition"]
  df["experiment"] = i
  tests = pd.concat([tests, df], ignore_index=True)

# L4 Q1: 

I have 19 experiments I want to append into one data frame, how do I do this?
The csv urls are given

* `pd.concat`

In [3]:
# Code Cell for L1 Q1

csv_urls = ["https://raw.githubusercontent.com/wesleybeckner/"\
                   "ds_for_engineers/main/data/mill_tool_wear/"\
                   "experiment_0{}.csv".format(i) for i in range(1,10)] + \
["https://raw.githubusercontent.com/wesleybeckner/"\
                   "ds_for_engineers/main/data/mill_tool_wear/"\
                   "experiment_{}.csv".format(i) for i in range(10,19)]                    

In [5]:
tests = pd.DataFrame()
for i, url in enumerate(csv_urls):
  df = pd.read_csv(url)
  df["feedrate"] = status.loc[i,"feedrate"]
  df["clamp_pressure"] = status.loc[i,"clamp_pressure"]
  df["tool_condition"] = status.loc[i,"tool_condition"]
  df["experiment"] = i
  tests = pd.concat([tests, df], ignore_index=True)

In [6]:
tests.head()

,X1_ActualPosition,X1_ActualVelocity,X1_ActualAcceleration,X1_CommandPosition,X1_CommandVelocity,X1_CommandAcceleration,X1_CurrentFeedback,X1_DCBusVoltage,X1_OutputCurrent,X1_OutputVoltage,X1_OutputPower,Y1_ActualPosition,Y1_ActualVelocity,Y1_ActualAcceleration,Y1_CommandPosition,Y1_CommandVelocity,Y1_CommandAcceleration,Y1_CurrentFeedback,Y1_DCBusVoltage,Y1_OutputCurrent,Y1_OutputVoltage,Y1_OutputPower,Z1_ActualPosition,Z1_ActualVelocity,Z1_ActualAcceleration,Z1_CommandPosition,Z1_CommandVelocity,Z1_CommandAcceleration,Z1_CurrentFeedback,Z1_DCBusVoltage,Z1_OutputCurrent,Z1_OutputVoltage,S1_ActualPosition,S1_ActualVelocity,S1_ActualAcceleration,S1_CommandPosition,S1_CommandVelocity,S1_CommandAcceleration,S1_CurrentFeedback,S1_DCBusVoltage,S1_OutputCurrent,S1_OutputVoltage,S1_OutputPower,S1_SystemInertia,M1_CURRENT_PROGRAM_NUMBER,M1_sequence_number,M1_CURRENT_FEEDRATE,Machining_Process,feedrate,clamp_pressure,tool_condition,experiment
0,198.0,0.0,0.00,198.0,0.0,0.000000,0.18,0.0207,329.0,2.77,-0.000001,158.0,-0.025,-6.25,158.0,0.0,0.000000,0.539,0.0167,328.0,1.84,6.430000e-07,119.0,0.0,0.00,119.0,0.0,0.000000,0.0,0.0,0.0,0.0,-361.0,0.001,0.250,-361.0,0.0,0.0,0.524,2.740000e-19,329.0,0.0,6.960000e-07,12.0,1.0,0.0,50.0,Starting,6,4.0,unworn,0
1,198.0,-10.8,-350.00,198.0,-13.6,-358.000000,-10.90,0.1860,328.0,23.30,0.004480,158.0,-19.800,-750.00,157.0,-24.6,-647.000000,-14.500,0.2810,325.0,37.80,1.260000e-02,119.0,-20.3,-712.00,118.0,-25.6,-674.000000,0.0,0.0,0.0,0.0,-361.0,0.000,0.250,-361.0,0.0,0.0,-0.288,2.740000e-19,328.0,0.0,-5.270000e-07,12.0,1.0,4.0,50.0,Prep,6,4.0,unworn,0
2,196.0,-17.8,-6.25,196.0,-17.9,-0.000095,-8.59,0.1400,328.0,30.60,0.005330,154.0,-32.500,0.00,154.0,-32.3,-0.000095,-7.790,0.1390,327.0,49.40,9.430000e-03,115.0,-33.7,37.50,115.0,-33.7,-0.000095,0.0,0.0,0.0,0.0,-361.0,0.000,-0.438,-361.0,0.0,0.0,0.524,2.740000e-19,328.0,0.0,9.100000e-07,12.0,1.0,7.0,50.0,Prep,6,4.0,unworn,0
3,194.0,-18.0,0.00,194.0,-17.9,-0.000095,-6.11,0.1300,327.0,30.30,0.004890,151.0,-32.600,-62.50,151.0,-32.3,-0.000095,-8.130,0.1560,325.0,47.60,1.050000e-02,112.0,-33.7,-6.25,112.0,-33.7,0.000000,0.0,0.0,0.0,0.0,-361.0,-0.001,-0.250,-361.0,0.0,0.0,-0.180,2.740000e-19,328.0,0.0,1.070000e-06,12.0,1.0,7.0,50.0,Prep,6,4.0,unworn,0
4,193.0,-17.9,-18.80,192.0,-17.9,0.000095,-5.70,0.1140,328.0,30.50,0.004250,148.0,-32.000,138.00,148.0,-32.3,0.000095,-13.800,0.2020,326.0,47.10,1.350000e-02,109.0,-33.6,18.80,108.0,-33.7,0.000000,0.0,0.0,0.0,0.0,-361.0,0.000,-0.188,-361.0,0.0,0.0,0.226,2.740000e-19,328.0,0.0,1.960000e-06,12.0,1.0,7.0,50.0,Prep,6,4.0,unworn,0


# L4 Q2:

The following two dataframes contain the same information (apart form the EBITDA/KG data). Start with one and end at the other.


In [7]:
margin = pd.read_csv('https://raw.githubusercontent.com/wesleybeckner/'\
        'ds_for_engineers/main/data/truffle_margin/truffle_margin_customer.csv')

orders = pd.read_csv('https://raw.githubusercontent.com/wesleybeckner/'\
                 'ds_for_engineers/main/data/truffle_margin/truffle_orders.csv')
time_cols = [i for i in orders.columns if '/' in i]

In [11]:
orders.head()

,Base Cake,Truffle Type,Primary Flavor,Secondary Flavor,Color Group,Customer,1/2020,2/2020,3/2020,4/2020,5/2020,6/2020,7/2020,8/2020,9/2020,10/2020,11/2020,12/2020
0,Butter,Candy Outer,Butter Pecan,Toffee,Taupe,Slugworth,53770.342593,40735.108025,40735.108025,40735.108025,53770.342593,40735.108025,40735.108025,40735.108025,53770.342593,40735.108025,40735.108025,40735.108025
1,Butter,Candy Outer,Ginger Lime,Banana,Amethyst,Slugworth,466477.578125,299024.088542,466477.578125,299024.088542,466477.578125,299024.088542,466477.578125,299024.088542,466477.578125,299024.088542,466477.578125,299024.088542
2,Butter,Candy Outer,Ginger Lime,Banana,Burgundy,Perk-a-Cola,80801.728070,51795.979532,51795.979532,51795.979532,80801.728070,51795.979532,51795.979532,51795.979532,80801.728070,51795.979532,51795.979532,51795.979532
3,Butter,Candy Outer,Ginger Lime,Banana,White,Fickelgruber,18046.111111,13671.296296,13671.296296,13671.296296,18046.111111,13671.296296,13671.296296,13671.296296,18046.111111,13671.296296,13671.296296,13671.296296
4,Butter,Candy Outer,Ginger Lime,Rum,Amethyst,Fickelgruber,19147.454268,12274.009146,12274.009146,12274.009146,12274.009146,12274.009146,19147.454268,12274.009146,12274.009146,12274.009146,12274.009146,12274.009146


In [9]:
margin.head()

,Base Cake,Truffle Type,Primary Flavor,Secondary Flavor,Color Group,Customer,Date,KG,EBITDA/KG
0,Butter,Candy Outer,Butter Pecan,Toffee,Taupe,Slugworth,1/2020,53770.342593,0.500424
1,Butter,Candy Outer,Ginger Lime,Banana,Amethyst,Slugworth,1/2020,466477.578125,0.220395
2,Butter,Candy Outer,Ginger Lime,Banana,Burgundy,Perk-a-Cola,1/2020,80801.728070,0.171014
3,Butter,Candy Outer,Ginger Lime,Banana,White,Fickelgruber,1/2020,18046.111111,0.233025
4,Butter,Candy Outer,Ginger Lime,Rum,Amethyst,Fickelgruber,1/2020,19147.454268,0.480689


In [12]:
# Code Cell for L1 Q2
orders.melt(id_vars=['Base Cake','Truffle Type','Primary Flavor','Secondary Flavor','Color Group','Customer'], 
           var_name='date', value_name='KG')


,Base Cake,Truffle Type,Primary Flavor,Secondary Flavor,Color Group,Customer,date,KG
0,Butter,Candy Outer,Butter Pecan,Toffee,Taupe,Slugworth,1/2020,53770.342593
1,Butter,Candy Outer,Ginger Lime,Banana,Amethyst,Slugworth,1/2020,466477.578125
2,Butter,Candy Outer,Ginger Lime,Banana,Burgundy,Perk-a-Cola,1/2020,80801.728070
3,Butter,Candy Outer,Ginger Lime,Banana,White,Fickelgruber,1/2020,18046.111111
4,Butter,Candy Outer,Ginger Lime,Rum,Amethyst,Fickelgruber,1/2020,19147.454268
...,...,...,...,...,...,...,...,...
1663,Tiramisu,Chocolate Outer,Doughnut,Pear,Amethyst,Fickelgruber,12/2020,38128.802589
1664,Tiramisu,Chocolate Outer,Doughnut,Pear,Burgundy,Zebrabar,12/2020,108.642857
1665,Tiramisu,Chocolate Outer,Doughnut,Pear,Teal,Zebrabar,12/2020,3517.933333
1666,Tiramisu,Chocolate Outer,Doughnut,Rock and Rye,Amethyst,Slugworth,12/2020,10146.898432


# L1 Q3:

Using the dataframe you create in Q1, create new X, y data that is based on a sweeping window size of 100 time steps

In [23]:
X = tests.drop(columns=['tool_condition', 'Machining_Process', 'experiment'],axis=1)
y = np.array(tests['tool_condition'])

The bellow runs very slow so I just break of after 50 iterations on each experiment

In [54]:
# Code Cell for L1 Q3
window = 100
exps = {}
for exp in tests.experiment.unique():
  subdf = tests.loc[tests.experiment == exp]
  subdf = subdf.drop(columns=['tool_condition', 'Machining_Process', 'experiment'],axis=1)
  print(exp)
  for i in range(subdf.shape[0]-window):
    feat = subdf.iloc[i:i+window,:].values.reshape(1,-1)
    if i == 0:
      new = feat
    else:
      new = np.vstack([new, feat])
    if i > 50:
      break
  exps[exp] = new 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


In [60]:
exps

{0: array([[ 198.  ,    0.  ,    0.  , ...,    6.  ,    6.  ,    4.  ],
        [ 198.  ,  -10.8 , -350.  , ...,    6.  ,    6.  ,    4.  ],
        [ 196.  ,  -17.8 ,   -6.25, ...,    6.  ,    6.  ,    4.  ],
        ...,
        [ 142.  ,   -2.7 ,  -18.8 , ...,    6.  ,    6.  ,    4.  ],
        [ 142.  ,   -2.03,   43.8 , ...,    6.  ,    6.  ,    4.  ],
        [ 142.  ,   -1.45,  -68.7 , ...,    6.  ,    6.  ,    4.  ]]),
 1: array([[198.,   0.,   0., ...,   6.,  20.,   4.],
        [198.,   0.,   0., ...,   6.,  20.,   4.],
        [198.,   0.,   0., ...,   6.,  20.,   4.],
        ...,
        [198.,   0.,   0., ...,  20.,  20.,   4.],
        [198.,   0.,   0., ...,  20.,  20.,   4.],
        [198.,   0.,   0., ...,  20.,  20.,   4.]]),
 2: array([[ 1.98e+02, -2.50e-02, -1.25e+01, ...,  6.00e+00,  6.00e+00,
          3.00e+00],
        [ 1.98e+02,  2.50e-02,  1.88e+01, ...,  6.00e+00,  6.00e+00,
          3.00e+00],
        [ 1.98e+02,  2.50e-02,  6.25e+00, ...,  6.00e+00,  6.